In [124]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os


# Set up the driver
os.environ['PATH'] += r'C:\Selenium drivers\chromedriver-win64\chromedriver-win64'
driver = webdriver.Chrome()

In [125]:
driver.get('https://www.nseindia.com/companies-listing/corporate-filings-financial-results')
new_session = True

In [126]:
# All the companies in NSE
import pandas as pd
df = pd.read_csv('../src/nse.csv')
with open('../src/info.txt', 'r') as file:
    date = file.read()
print("Companies in nse as of" , date ,"are", len(df['SYMBOL \n']))
# Delete every column of the dataframe except the symbol
df = df['SYMBOL \n']
print("First company", df[0])

Companies in nse as of 23-May-2024 are 1730
First company AUSOMENT


In [127]:

def select_custom_date():
    custom_date = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='JavaScript:;' and @data-val='Custom']")))
    custom_date.click()
    buttons = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "button.btn.btn-outline-secondary.border-left-0")))
    from_date_button = buttons[0]
    # from_date_button.click()


    # choosing the date
    period_div = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div[data-role="period"]')))
    time.sleep(1)
    period_div.click()
    # period_div.click()
    # period_div.click()

    # Replace 'driver' with your WebDriver instance
    wait = WebDriverWait(driver, 10)
    chevron_div = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div > i.fa.fa-chevron-left')))
    chevron_div_parent = chevron_div.find_element(By.XPATH, '..')

    # Click the parent div
    for i in range(24):
        chevron_div_parent.click()


    # Check the month and the date
    wait = WebDriverWait(driver, 10)

    # Find the <div> element with the text "Jan"
    jan_div = wait.until(EC.presence_of_element_located((By.XPATH, '//table//tbody/tr/td/div[@role="button" and text()="Jan"]')))

    # Click the "Jan" <div> element
    jan_div.click()


    wait = WebDriverWait(driver, 20)

    # Find the <div> element with the text "1"
    day_one_div = wait.until(EC.presence_of_element_located((By.XPATH, '//table//tbody/tr/td/div[@role="button" and text()="1"]')))

    # Click the "1" <div> element
    day_one_div.click()

    time.sleep(0.2)
    # get today's date
    today_year = time.strftime("%Y")
    today_month_ = time.strftime("%m")
    month_name = {
        '01': 'Jan',
        '02': 'Feb',
        '03': 'Mar',
        '04': 'Apr',
        '05': 'May',
        '06': 'Jun',
        '07': 'Jul',
        '08': 'Aug',
        '09': 'Sep',
        '10': 'Oct',
        '11': 'Nov',
        '12': 'Dec'
    }
    today_month = month_name[today_month_]
    today_day = time.strftime("%d")

    print(today_year, today_month, today_day)

    to_date_button = buttons[1]
    to_date_button.click()
    # Find the <div> element with the text "today_day"
    time.sleep(1)
    wait = WebDriverWait(driver, 10)

    # Find the second <div> element with the text "1" - date 1
    day_two_divs = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//table//tbody/tr/td[@class="current-month gj-cursor-pointer"]/div[@role="button" and text()="1"]')))
    if len(day_two_divs) >= 2:
        second_day_two_div = day_two_divs[1]
        second_day_two_div.click()
    else:
        print("Less than two '2' elements found.")
        second_day_two_div = day_two_divs[0]
        second_day_two_div.click()
    
    print("here")
    go_button = driver.find_element(By.XPATH, '//button[@class="btn flat-button-white w-100 filterbtn" and text()="GO"]')
    # go_button = wait.until(EC.presence_of_element_located((By.XPATH, '//button[@class="btn flat-button-white w-100 filterbtn active"]')))
    go_button.click()
    

In [128]:
def company_data_download(company_name, new_session):
    company = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "financials_equities_companyName")))
    period_ended = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "financials_equities_period")))
    
    # for option in period_ended.find_elements(By.TAG_NAME, "option"):
    #     if option.text == "Annual":
    #         option.click()
    #         break
    time.sleep(3)
    company.send_keys(company_name)
    # TODO: briefly wait for the autocomplete to appear
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "lt")))
    company.send_keys(Keys.ARROW_DOWN)
    company.send_keys(Keys.ENTER)
    
    if new_session:
        select_custom_date()

    # Click the download link
    time.sleep(3)
    download_csv = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "CFfinancialequity-download")))
    download_csv.click()
    

In [129]:
# make a new text file to store the companies that have been downloaded
count = 0
for symbol in df:
    # if symbol already exists in the downloaded_companies.txt file, skip it
    with open('../src/downloaded_companies.txt', 'r') as file:
        if symbol in file.read():
            continue
    company_data_download(symbol, new_session)
    new_session = False
    with open('../src/downloaded_companies.txt', 'a') as file:
        file.write(symbol + '\n')
    time.sleep(1)
    count += 1

print("Downloaded", count, "companies")

2024 May 23
Less than two '2' elements found.
here


ElementClickInterceptedException: Message: element click intercepted: Element <a id="CFfinancialequity-download" data-url="" href="javascript:downloadCSV('CFfinancialequity-download')">...</a> is not clickable at point (78, 650). Other element would receive the click: <span class="lt">...</span>
  (Session info: chrome=125.0.6422.77)
Stacktrace:
	GetHandleVerifier [0x00007FF766C11F52+60322]
	(No symbol) [0x00007FF766B8CEC9]
	(No symbol) [0x00007FF766A47EBA]
	(No symbol) [0x00007FF766A9F32E]
	(No symbol) [0x00007FF766A9CCF2]
	(No symbol) [0x00007FF766A9A18B]
	(No symbol) [0x00007FF766A99356]
	(No symbol) [0x00007FF766A8B491]
	(No symbol) [0x00007FF766ABC21A]
	(No symbol) [0x00007FF766A8ADB6]
	(No symbol) [0x00007FF766ABC430]
	(No symbol) [0x00007FF766ADBC80]
	(No symbol) [0x00007FF766ABBFC3]
	(No symbol) [0x00007FF766A89617]
	(No symbol) [0x00007FF766A8A211]
	GetHandleVerifier [0x00007FF766F2949D+3301613]
	GetHandleVerifier [0x00007FF766F736C3+3605267]
	GetHandleVerifier [0x00007FF766F6

In [ ]:
driver.quit()